In [1]:

import datetime
import numpy as np
import cv2
from itertools import cycle
import pickle
import pathlib
import math
import tqdm
import scipy.io
from matplotlib import pyplot as plt
import scipy.io
import h5py
import re
from lxml import etree as ET
import scipy.signal as sig
import pandas as pd
from scipy.stats import kde
from BlockSync_current import BlockSync
import UtilityFunctions_newOE as uf
from scipy import signal
import bokeh
import seaborn as sns
from matplotlib import rcParams
%matplotlib inline
plt.style.use('default')
rcParams['pdf.fonttype'] = 42  # Ensure fonts are embedded and editable
rcParams['ps.fonttype'] = 42  # Ensure compatibility with vector outputs


def bokeh_plotter(data_list, x_axis_list=None, label_list=None,
                  plot_name='default',
                  x_axis_label='X', y_axis_label='Y',
                  peaks=None, peaks_list=False, export_path=False):
    """Generates an interactive Bokeh plot for the given data vector.
    Args:
        data_list (list or array): The data to be plotted.
        label_list (list of str): The labels of the data vectors
        plot_name (str, optional): The title of the plot. Defaults to 'default'.
        x_axis (str, optional): The label for the x-axis. Defaults to 'X'.
        y_axis (str, optional): The label for the y-axis. Defaults to 'Y'.
        peaks (list or array, optional): Indices of peaks to highlight on the plot. Defaults to None.
        export_path (False or str): when set to str, will output the resulting html fig
    """
    color_cycle = cycle(bokeh.palettes.Category10_10)
    fig = bokeh.plotting.figure(title=f'bokeh explorer: {plot_name}',
                                x_axis_label=x_axis_label,
                                y_axis_label=y_axis_label,
                                plot_width=1500,
                                plot_height=700)

    for i, data_vector in enumerate(data_list):

        color = next(color_cycle)

        if x_axis_list is None:
            x_axis = range(len(data_vector))
        elif len(x_axis_list) == len(data_list):
            print('x_axis manually set')
            x_axis = x_axis_list[i]
        else:
            raise Exception(
                'problem with x_axis_list input - should be either None, or a list with the same length as data_list')
        if label_list is None:
            fig.line(x_axis, data_vector, line_color=color, legend_label=f"Line {i + 1}")
        elif len(label_list) == len(data_list):
            fig.line(range(len(data_vector)), data_vector, line_color=color, legend_label=f"{label_list[i]}")
        if peaks is not None and peaks_list is True:
            fig.circle(peaks[i], data_vector[peaks[i]], size=10, color=color)

    if peaks is not None and peaks_list is False:
        fig.circle(peaks, data_vector[peaks], size=10, color='red')

    if export_path is not False:
        print(f'exporting to {export_path}')
        bokeh.io.output.output_file(filename=str(export_path / f'{plot_name}.html'), title=f'{plot_name}')
    bokeh.plotting.show(fig)


def load_eye_data_2d_w_rotation_matrix(block):
    """
    This function checks if the eye dataframes and rotation dict object exist, then imports them
    :param block: The current blocksync class with verifiec re/le dfs
    :return: None
    """
    try:
        block.left_eye_data = pd.read_csv(block.analysis_path / 'left_eye_data.csv', index_col=0, engine='python')
        block.right_eye_data = pd.read_csv(block.analysis_path / 'right_eye_data.csv', index_col=0, engine='python')
    except FileNotFoundError:
        print('eye_data files not found, run the pipeline!')
        return

    try:
        with open(block.analysis_path / 'rotate_eye_data_params.pkl', 'rb') as file:
            rotation_dict = pickle.load(file)
            block.left_rotation_matrix = rotation_dict['left_rotation_matrix']
            block.right_rotation_matrix = rotation_dict['right_rotation_matrix']
            block.left_rotation_angle = rotation_dict['left_rotation_angle']
            block.right_rotation_angle = rotation_dict['right_rotation_angle']
    except FileNotFoundError:
        print('No rotation matrix file, create it')


def create_saccade_events_df(eye_data_df, speed_threshold, bokeh_verify_threshold=False, magnitude_calib=1,
                             speed_profile=True):
    """
    Detects saccade events in eye tracking data and computes relevant metrics.

    Parameters:
    - eye_data_df (pd.DataFrame): Input DataFrame containing eye tracking data.
    - speed_threshold (float): Threshold for saccade detection based on speed.

    Returns:
    - df (pd.DataFrame): Modified input DataFrame with added columns for speed and saccade detection.
    - saccade_events_df (pd.DataFrame): DataFrame containing information about detected saccade events.

    Steps:
    1. Calculate speed components ('speed_x', 'speed_y') based on differences in 'center_x' and 'center_y'.
    2. Compute the magnitude of the velocity vector ('speed_r').
    3. Create a binary column ('is_saccade') indicating saccade events based on the speed threshold.
    4. Determine saccade onset and offset indices and timestamps.
    5. Create a DataFrame ('saccade_events_df') with columns:
        - 'saccade_start_ind': Indices of saccade onset.
        - 'saccade_start_timestamp': Timestamps corresponding to saccade onset.
        - 'saccade_end_ind': Indices of saccade offset.
        - 'saccade_end_timestamp': Timestamps corresponding to saccade offset.
        - 'length': Duration of each saccade event.
    6. Calculate distance traveled and angles for each saccade event.
    7. Append additional columns to 'saccade_events_df':
        - 'magnitude': Magnitude of the distance traveled during each saccade.
        - 'angle': Angle of the saccade vector in degrees.
        - 'initial_x', 'initial_y': Initial coordinates of the saccade.
        - 'end_x', 'end_y': End coordinates of the saccade.

    Note: The original 'eye_data_df' is not modified; modified data is returned as 'df'.
    """
    df = eye_data_df
    df['speed_x'] = df['center_x'].diff()  # Difference between consecutive 'center_x' values
    df['speed_y'] = df['center_y'].diff()  # Difference between consecutive 'center_y' values

    # Step 2: Calculate magnitude of the velocity vector (R vector speed)
    df['speed_r'] = (df['speed_x'] ** 2 + df['speed_y'] ** 2) ** 0.5

    # Create a column for saccade detection
    df['is_saccade'] = df['speed_r'] > speed_threshold

    # create a saccade_on_off indicator where 1 is rising edge and -1 is falling edge by subtracting a shifted binary mask
    saccade_on_off = df.is_saccade.astype(int) - df.is_saccade.shift(periods=1, fill_value=False).astype(int)
    saccade_on_inds = np.where(saccade_on_off == 1)[
                          0] - 1  # notice the manual shift here, chosen to include the first (sometimes slower) eye frame, just before saccade threshold crossing
    saccade_on_ms = df['ms_axis'].iloc[saccade_on_inds]
    saccade_on_timestamps = df['OE_timestamp'].iloc[saccade_on_inds]
    saccade_off_inds = np.where(saccade_on_off == -1)[0]
    saccade_off_timestamps = df['OE_timestamp'].iloc[saccade_off_inds]
    saccade_off_ms = df['ms_axis'].iloc[saccade_off_inds]

    saccade_dict = {'saccade_start_ind': saccade_on_inds,
                    'saccade_start_timestamp': saccade_on_timestamps.values,
                    'saccade_end_ind': saccade_off_inds,
                    'saccade_end_timestamp': saccade_off_timestamps.values,
                    'saccade_on_ms': saccade_on_ms.values,
                    'saccade_off_ms': saccade_off_ms.values}

    saccade_events_df = pd.DataFrame.from_dict(saccade_dict)
    saccade_events_df['length'] = saccade_events_df['saccade_end_ind'] - saccade_events_df['saccade_start_ind']
    # Drop columns used for intermediate steps
    df = df.drop(['is_saccade'], axis=1)

    distances = []
    angles = []
    speed_list = []
    diameter_list = []
    for index, row in tqdm.tqdm(saccade_events_df.iterrows()):
        saccade_samples = df.loc[(df['OE_timestamp'] >= row['saccade_start_timestamp']) &
                                 (df['OE_timestamp'] <= row['saccade_end_timestamp'])]
        distance_traveled = saccade_samples['speed_r'].sum()
        if speed_profile:
            saccade_speed_profile = saccade_samples['speed_r'].values
            speed_list.append(saccade_speed_profile)
        saccade_diameter_profile = saccade_samples['pupil_diameter'].values
        diameter_list.append(saccade_diameter_profile)
        # Calculate angle from initial position to endpoint
        initial_position = saccade_samples.iloc[0][['center_x', 'center_y']]
        endpoint = saccade_samples.iloc[-1][['center_x', 'center_y']]
        overall_angle = np.arctan2(endpoint['center_y'] - initial_position['center_y'],
                                   endpoint['center_x'] - initial_position['center_x'])

        angles.append(overall_angle)
        distances.append(distance_traveled)

    saccade_events_df['magnitude_raw'] = np.array(distances)
    saccade_events_df['magnitude'] = np.array(distances) * magnitude_calib
    saccade_events_df['angle'] = np.where(np.isnan(angles), angles, np.rad2deg(
        angles) % 360)  # Convert radians to degrees and ensure result is in [0, 360)
    start_ts = saccade_events_df['saccade_start_timestamp'].values
    end_ts = saccade_events_df['saccade_end_timestamp'].values
    saccade_start_df = df[df['OE_timestamp'].isin(start_ts)]
    saccade_end_df = df[df['OE_timestamp'].isin(end_ts)]
    start_x_coord = saccade_start_df['center_x']
    start_y_coord = saccade_start_df['center_y']
    end_x_coord = saccade_end_df['center_x']
    end_y_coord = saccade_end_df['center_y']
    saccade_events_df['initial_x'] = start_x_coord.values
    saccade_events_df['initial_y'] = start_y_coord.values
    saccade_events_df['end_x'] = end_x_coord.values
    saccade_events_df['end_y'] = end_y_coord.values
    saccade_events_df['calib_dx'] = (saccade_events_df['end_x'].values - saccade_events_df[
        'initial_x'].values) * magnitude_calib
    saccade_events_df['calib_dy'] = (saccade_events_df['end_y'].values - saccade_events_df[
        'initial_y'].values) * magnitude_calib
    if speed_profile:
        saccade_events_df['speed_profile'] = speed_list
    saccade_events_df['diameter_profile'] = diameter_list
    if bokeh_verify_threshold:
        bokeh_plotter(data_list=[df.speed_r], label_list=['Pupil Velocity'], peaks=saccade_on_inds)

    return df, saccade_events_df


# create a multi-animal block_collection:

def create_block_collections(animals, block_lists, experiment_path, bad_blocks=None):
    """
    Create block collections and a block dictionary from multiple animals and their respective block lists.

    Parameters:
    - animals: list of str, names of the animals.
    - block_lists: list of lists of int, block numbers corresponding to each animal.
    - experiment_path: pathlib.Path, path to the experiment directory.
    - bad_blocks: list of int, blocks to exclude. Default is an empty list.

    Returns:
    - block_collection: list of BlockSync objects for all specified blocks.
    - block_dict: dictionary where keys are block numbers as strings and values are BlockSync objects.
    """
    import UtilityFunctions_newOE as uf

    if bad_blocks is None:
        bad_blocks = []

    block_collection = []
    block_dict = {}

    for animal, blocks in zip(animals, block_lists):
        # Generate blocks for the current animal
        current_blocks = uf.block_generator(
            block_numbers=blocks,
            experiment_path=experiment_path,
            animal=animal,
            bad_blocks=bad_blocks
        )
        # Add to collection and dictionary
        block_collection.extend(current_blocks)
        for b in current_blocks:
            block_dict[f"{animal}_block_{b.block_num}"] = b

    return block_collection, block_dict


In [2]:
# BLOCK DEFINITION #
# This was the previous run
#animals = ['PV_62', 'PV_126', 'PV_57']
#block_lists = [[24, 26, 38], [7, 8, 9, 10, 11, 12], [7, 8, 9, 12, 13]]
#This with new animals:
animals = ['PV_126']
block_lists = [[7]]
experiment_path = pathlib.Path(r"Z:\Nimrod\experiments")
bad_blocks = [0]  # Example of bad blocks

block_collection, block_dict = create_block_collections(
    animals=animals,
    block_lists=block_lists,
    experiment_path=experiment_path,
    bad_blocks=bad_blocks
)
for block in block_collection:
    block.parse_open_ephys_events()
    block.get_eye_brightness_vectors()
    block.synchronize_block()
    block.create_eye_brightness_df(threshold_value=20)

    # if the code fails here, go to manual synchronization
    block.import_manual_sync_df()
    block.read_dlc_data()
    block.calibrate_pixel_size(10)
    #load_eye_data_2d_w_rotation_matrix(block) #should be integrated again... later

    for block in block_collection:
        # block.left_eye_data = pd.read_csv(block.analysis_path / f'left_eye_data_corr_angles.csv')
        # block.right_eye_data = pd.read_csv(block.analysis_path / 'right_eye_data_corr_angles.csv')
        #block.left_eye_data = pd.read_csv(block.analysis_path / f'left_eye_data_degrees_raw_xflipped.csv')
        #block.right_eye_data = pd.read_csv(block.analysis_path / 'right_eye_data_degrees_raw_xflipped.csv')
        block.left_eye_data = pd.read_csv(block.analysis_path / f'left_eye_data_degrees_raw_verified.csv')
        block.right_eye_data = pd.read_csv(block.analysis_path / 'right_eye_data_degrees_raw_verified.csv')
        # block.left_eye_data = pd.read_csv(block.analysis_path / f'left_eye_data_3d_corr_verified.csv')
        # block.right_eye_data = pd.read_csv(block.analysis_path / 'right_eye_data_3d_corr_verified.csv')
        #block.left_eye_data = pd.read_csv(block.analysis_path / f'left_eye_data_degrees_rotated_verified.csv')
        #block.right_eye_data = pd.read_csv(block.analysis_path / 'right_eye_data_degrees_rotated_verified.csv')

    # calibrate pupil diameter:
    # if 'pupil_diameter' not in block.left_eye_data.columns:
    #     block.left_eye_data['pupil_diameter_pixels'] = block.left_eye_data.major_ax * 2 * np.pi
    #     block.right_eye_data['pupil_diameter_pixels'] = block.right_eye_data.major_ax * 2 * np.pi
    #     block.left_eye_data['pupil_diameter'] = block.left_eye_data['pupil_diameter_pixels'] * block.L_pix_size
    #     block.right_eye_data['pupil_diameter'] = block.right_eye_data['pupil_diameter_pixels'] * block.R_pix_size

instantiated block number 007 at Path: Z:\Nimrod\experiments\PV_126\2024_07_18\block_007, new OE version
Found the sample rate for block 007 in the xml file, it is 20000 Hz
created the .oe_rec attribute as an open ephys recording obj with get_data functionality
retrieving zertoh sample number for block 007
got it!
running parse_open_ephys_events...
block 007 has a parsed events file, reading...
Getting eye brightness values for block 007...
Found an existing file!
Eye brightness vectors generation complete.
blocksync_df loaded from analysis folder
eye_brightness_df loaded from analysis folder
eye dataframes loaded from analysis folder
got the calibration values from the analysis folder


In [13]:
for block in block_collection:
    block.handle_eye_videos()
    block.handle_arena_files()
    if 'pupil_diameter' not in block.left_eye_data.columns:
        print(f'calculating pupil diameter for {block} ')
        block.left_eye_data['pupil_diameter_pixels'] = block.left_eye_data.major_ax
        block.right_eye_data['pupil_diameter_pixels'] = block.right_eye_data.major_ax
        block.left_eye_data['pupil_diameter'] = block.left_eye_data['pupil_diameter_pixels'] * block.L_pix_size
        block.right_eye_data['pupil_diameter'] = block.right_eye_data['pupil_diameter_pixels'] * block.R_pix_size

handling eye video files
converting videos...
converting files: ['Z:\\Nimrod\\experiments\\PV_126\\2024_07_18\\block_007\\eye_videos\\LE\\wake3_640x480_60hz_experiment_1_recording_0\\wake3.h264', 'Z:\\Nimrod\\experiments\\PV_126\\2024_07_18\\block_007\\eye_videos\\RE\\wake3_640x480_60hz_experiment_1_recording_0\\wake3.h264'] 
 avoiding conversion on files: ['Z:\\Nimrod\\experiments\\PV_126\\2024_07_18\\block_007\\eye_videos\\LE\\wake3_640x480_60hz_experiment_1_recording_0\\wake3_LE.mp4', 'Z:\\Nimrod\\experiments\\PV_126\\2024_07_18\\block_007\\eye_videos\\RE\\wake3_640x480_60hz_experiment_1_recording_0\\wake3.mp4']
The file Z:\Nimrod\experiments\PV_126\2024_07_18\block_007\eye_videos\RE\wake3_640x480_60hz_experiment_1_recording_0\wake3.mp4 already exists, no conversion necessary
Validating videos...
The video named wake3_LE.mp4 has reported 121804 frames and has 121805 frames, it has dropped -1 frames
The video named wake3.mp4 has reported 121901 frames and has 121901 frames, it has dr

In [9]:
# if ms_axis is missing from final sync df:
block.final_sync_df['ms_axis'] = block.final_sync_df['Arena_TTL'].values / (block.sample_rate / 1000)

In [31]:
from __future__ import annotations

import os
from pathlib import Path
from typing import Optional, Sequence, Dict, Tuple, Union, Literal

import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm


def export_block_synchronized_montage_video_v3(
    block: object,
    start_ms: float,
    end_ms: float,
    out_path: os.PathLike | str,
    *,
    fps: float = 60.0,

    # Arena selection + manual patch
    arena_video: Optional[Union[int, str]] = None,  # None => prompt; int => index; str => substring match
    arena_frame_cols: Sequence[str] = (
        "Arena_frame", "arena_frame", "arena_frames", "arena_frame_idx",
        "frame", "frame_idx", "video_frame", "arena_idx"
    ),
    use_final_sync_df: bool = True,
    arena_frame_shift: int = 0,  # + shifts arena later (uses later frames); - shifts earlier

    # --- NEW: choose eye video source ---
    eye_video_mode: Literal["auto", "raw", "dlc"] = "auto",  # <-- requested flag
    dlc_name_hint: str = "DLC",  # substring to identify DLC files (case-insensitive)

    # Geometry / appearance
    top_banner_h: int = 60,
    trace_h: int = 220,
    fit_eyes_to_arena_height: bool = False,  # scale eyes to arena height (preserve AR)
    flip_eyes_vertical: bool = True,

    # Trace controls
    trace_window_ms: Optional[float] = None,
    normalize_traces: bool = False,
    robust_ylim_percentiles: Tuple[float, float] = (5.0, 95.0),
    trace_signals: Sequence[str] = ("pupil_diameter", "phi", "theta"),
    trace_col_map: Optional[Dict[str, str]] = None,  # e.g. {"theta":"k_theta","phi":"k_phi"}

    # Disqualification / missing frames
    disqualify_cols: Sequence[str] = ("center_x", "center_y"),
    show_disqualified_badge: bool = True,

    # Output encoding
    codec: str = "mp4v",
    timestamp_precision_ms: int = 0,

    show_debug_prints: bool = True,
) -> Path:
    """
    Export synchronized montage video:
        [Right eye | Arena | Left eye]
      + top text banner
      + bottom traces panel (pupil_diameter, phi, theta)

    NEW: eye_video_mode controls whether to use raw eye mp4s or DLC-annotated mp4s.
    """

    # -------------------------- helpers --------------------------
    def _require_attr(obj: object, name: str):
        if not hasattr(obj, name):
            raise AttributeError(f"block is missing required attribute '{name}'")
        return getattr(obj, name)

    def _require_cols(df: pd.DataFrame, cols: Sequence[str], df_name: str):
        missing = [c for c in cols if c not in df.columns]
        if missing:
            raise ValueError(f"{df_name} is missing required columns: {missing}")

    def _pick_first_video(paths, name: str) -> Path:
        if paths is None:
            raise ValueError(f"block.{name} is None")
        if isinstance(paths, (str, os.PathLike)):
            p = Path(paths)
            if not p.exists():
                raise FileNotFoundError(p)
            return p
        if isinstance(paths, (list, tuple)) and len(paths) > 0:
            p = Path(paths[0])
            if not p.exists():
                raise FileNotFoundError(p)
            return p
        raise ValueError(f"block.{name} has no usable video path(s)")

    def _open_cap(p: Path, label: str) -> cv2.VideoCapture:
        cap = cv2.VideoCapture(str(p))
        if not cap.isOpened():
            raise RuntimeError(f"Cannot open {label} video: {p}")
        return cap

    def _nearest_row_by_ms(df: pd.DataFrame, ms: float) -> pd.Series:
        arr = df["ms_axis"].to_numpy(dtype=float)
        if arr.size == 0:
            raise ValueError("ms_axis array is empty.")
        idx = int(np.nanargmin(np.abs(arr - float(ms))))
        return df.iloc[idx]

    def _resolve_arena_frame_col(fs: pd.DataFrame) -> str:
        for c in arena_frame_cols:
            if c in fs.columns:
                return c
        raise ValueError(f"final_sync_df has no recognizable arena frame column. Tried: {list(arena_frame_cols)}")

    def _safe_put_text(img, text, org, color, scale=0.6, thickness=2):
        x, y = org
        cv2.putText(img, text, (x + 1, y + 1), cv2.FONT_HERSHEY_SIMPLEX, scale, (0, 0, 0),
                    thickness + 2, cv2.LINE_AA)
        cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_SIMPLEX, scale, color, thickness, cv2.LINE_AA)

    def _make_banner(W: int, lines: Sequence[str]) -> np.ndarray:
        banner = np.zeros((top_banner_h, W, 3), dtype=np.uint8)
        y = 24
        for ln in lines:
            _safe_put_text(banner, ln, (12, y), (255, 255, 255), scale=0.7, thickness=2)
            y += 24
            if y > top_banner_h - 8:
                break
        return banner

    def _read_frame_at(cap: cv2.VideoCapture, frame_idx: int) -> Optional[np.ndarray]:
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(frame_idx))
        ok, frame = cap.read()
        return frame if ok else None

    def _clamp_idx(idx: Optional[int], cap: cv2.VideoCapture) -> Optional[int]:
        if idx is None:
            return None
        n = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
        if n > 0:
            return int(np.clip(int(idx), 0, n - 1))
        return max(0, int(idx))

    def _resize_to_height(img: np.ndarray, target_h: int) -> np.ndarray:
        h, w = img.shape[:2]
        if h == target_h:
            return img
        new_w = max(1, int(round(w * (target_h / float(h)))))
        return cv2.resize(img, (new_w, target_h), interpolation=cv2.INTER_AREA)

    def _robust_limits(x: np.ndarray, p_lo: float, p_hi: float) -> Tuple[float, float]:
        x = np.asarray(x, dtype=float)
        x = x[np.isfinite(x)]
        if x.size < 2:
            return (-1.0, 1.0)
        lo = np.percentile(x, p_lo)
        hi = np.percentile(x, p_hi)
        if not np.isfinite(lo) or not np.isfinite(hi) or abs(hi - lo) < 1e-12:
            lo, hi = float(np.min(x)), float(np.max(x))
        if abs(hi - lo) < 1e-12:
            lo -= 1.0
            hi += 1.0
        return float(lo), float(hi)

    def _map_to_axis(vals: np.ndarray, lo: float, hi: float) -> np.ndarray:
        return (vals - lo) / (hi - lo + 1e-12)

    def _draw_trace_panel(
        W: int,
        t_ms: float,
        t_grid: np.ndarray,
        Lsig: Dict[str, np.ndarray],
        Rsig: Dict[str, np.ndarray],
        t0: float,
        t1: float,
    ) -> np.ndarray:
        panel = np.zeros((trace_h, W, 3), dtype=np.uint8)

        if trace_window_ms is None:
            w0, w1 = t0, t1
        else:
            half = float(trace_window_ms) / 2.0
            w0, w1 = max(t0, t_ms - half), min(t1, t_ms + half)
            if w1 <= w0:
                w0, w1 = t0, t1

        idx = np.where((t_grid >= w0) & (t_grid <= w1))[0]
        if idx.size < 2:
            return panel

        tg = t_grid[idx]
        x = (tg - w0) / (w1 - w0 + 1e-12)
        xpix = (x * (W - 1)).astype(int)

        n_axes = len(trace_signals)
        pad_y = 10
        axis_h = max(40, (trace_h - 2 * pad_y) // max(1, n_axes))
        axes = []
        for j, name in enumerate(trace_signals):
            y0 = pad_y + j * axis_h
            y1 = min(trace_h - pad_y, y0 + axis_h) - 8
            axes.append((name, y0, y1))

        cursor_x = int(round((np.clip(t_ms, w0, w1) - w0) / (w1 - w0 + 1e-12) * (W - 1)))
        cv2.line(panel, (cursor_x, 0), (cursor_x, trace_h - 1), (120, 120, 120), 1)

        _safe_put_text(panel, f"t = {t_ms:.{timestamp_precision_ms}f} ms", (12, 22),
                       (255, 255, 255), scale=0.65, thickness=2)

        color_L = (80, 220, 80)    # green
        color_R = (220, 220, 80)   # yellow

        for name, y0, y1 in axes:
            cv2.rectangle(panel, (0, y0), (W - 1, y1), (20, 20, 20), 1)
            L = Lsig.get(name, None)
            R = Rsig.get(name, None)
            if L is None or R is None:
                continue

            Lw = L[idx].astype(float)
            Rw = R[idx].astype(float)

            yy0, yy1 = int(y0 + 18), int(y1 - 8)
            Hax = max(2, yy1 - yy0)

            if normalize_traces:
                # normalize per-window, per-eye
                Llo, Lhi = np.nanmin(Lw), np.nanmax(Lw)
                Rlo, Rhi = np.nanmin(Rw), np.nanmax(Rw)
                Ln = np.full_like(Lw, 0.5) if (not np.isfinite(Llo) or not np.isfinite(Lhi) or abs(Lhi - Llo) < 1e-12) else _map_to_axis(Lw, Llo, Lhi)
                Rn = np.full_like(Rw, 0.5) if (not np.isfinite(Rlo) or not np.isfinite(Rhi) or abs(Rhi - Rlo) < 1e-12) else _map_to_axis(Rw, Rlo, Rhi)
                label = f"{name} (norm)"
            else:
                # shared robust limits (both eyes) in window
                p_lo, p_hi = robust_ylim_percentiles
                lo, hi = _robust_limits(np.concatenate([Lw, Rw]), p_lo, p_hi)
                Ln = _map_to_axis(Lw, lo, hi)
                Rn = _map_to_axis(Rw, lo, hi)
                label = f"{name} [{lo:.2f},{hi:.2f}]"

            yL = (yy0 + (1.0 - np.clip(Ln, 0, 1)) * (Hax - 1)).astype(int)
            yR = (yy0 + (1.0 - np.clip(Rn, 0, 1)) * (Hax - 1)).astype(int)

            for k in range(1, len(xpix)):
                if np.isfinite(yL[k - 1]) and np.isfinite(yL[k]):
                    cv2.line(panel, (xpix[k - 1], yL[k - 1]), (xpix[k], yL[k]), color_L, 1, cv2.LINE_AA)
                if np.isfinite(yR[k - 1]) and np.isfinite(yR[k]):
                    cv2.line(panel, (xpix[k - 1], yR[k - 1]), (xpix[k], yR[k]), color_R, 1, cv2.LINE_AA)

            _safe_put_text(panel, label, (12, y0 + 16), (200, 200, 200), scale=0.55, thickness=1)
            _safe_put_text(panel, "L", (W - 60, y0 + 16), color_L, scale=0.55, thickness=1)
            _safe_put_text(panel, "R", (W - 35, y0 + 16), color_R, scale=0.55, thickness=1)

        return panel

    def _choose_arena_video(arena_list: Sequence[str], choice: Optional[Union[int, str]]) -> Path:
        paths = [Path(p) for p in arena_list]
        if not paths:
            raise ValueError("block.arena_videos is empty.")
        if isinstance(choice, int):
            if choice < 0 or choice >= len(paths):
                raise ValueError(f"arena_video index {choice} out of range (0..{len(paths)-1})")
            return paths[int(choice)]
        if isinstance(choice, str) and choice.strip():
            key = choice.strip().lower()
            hits = [p for p in paths if key in p.name.lower()]
            if not hits:
                raise ValueError(f"arena_video='{choice}' did not match any arena video name.")
            hits.sort(key=lambda p: len(p.name))
            return hits[0]
        # prompt
        print("\nSelect arena video:")
        for i, p in enumerate(paths):
            print(f"  [{i}] {p.name}")
        raw = ""
        try:
            raw = input("Enter arena index (default 0): ").strip()
        except Exception:
            raw = ""
        idx = 0 if raw == "" else int(raw)
        if idx < 0 or idx >= len(paths):
            raise ValueError(f"arena index {idx} out of range.")
        return paths[idx]

    def _resolve_eye_video(raw_path: Path, mode: str) -> Path:
        """
        raw_path: block.le_videos[0] or block.re_videos[0]
        mode: 'raw' | 'dlc' | 'auto'
        Finds DLC mp4s in the same folder if requested.
        """
        raw_path = Path(raw_path)
        if mode == "raw":
            return raw_path

        folder = raw_path.parent
        if not folder.exists():
            if mode == "dlc":
                raise FileNotFoundError(f"Eye video folder not found: {folder}")
            return raw_path

        # all mp4 with DLC hint
        hint = dlc_name_hint.lower()
        dlc_candidates = [p for p in folder.glob("*.mp4") if hint in p.name.lower()]

        # prefer candidates that contain the raw stem
        stem = raw_path.stem.lower()
        stem_hits = [p for p in dlc_candidates if stem in p.stem.lower()]
        candidates = stem_hits if stem_hits else dlc_candidates

        if candidates:
            # stable pick
            candidates.sort(key=lambda p: p.name)
            return candidates[0]

        if mode == "dlc":
            raise FileNotFoundError(
                f"eye_video_mode='dlc' but no DLC mp4 found in {folder} (hint='{dlc_name_hint}')"
            )
        return raw_path

    # -------------------------- validation --------------------------
    start_ms = float(start_ms)
    end_ms = float(end_ms)
    if end_ms <= start_ms:
        raise ValueError(f"end_ms must be > start_ms (got start_ms={start_ms}, end_ms={end_ms})")

    left_df: pd.DataFrame = _require_attr(block, "left_eye_data")
    right_df: pd.DataFrame = _require_attr(block, "right_eye_data")
    _require_cols(left_df, ["ms_axis", "eye_frame"], "left_eye_data")
    _require_cols(right_df, ["ms_axis", "eye_frame"], "right_eye_data")

    fs = None
    arena_fcol = None
    if use_final_sync_df:
        fs = _require_attr(block, "final_sync_df")
        if not isinstance(fs, pd.DataFrame):
            raise ValueError("block.final_sync_df is not a pandas DataFrame.")
        _require_cols(fs, ["ms_axis"], "final_sync_df")
        arena_fcol = _resolve_arena_frame_col(fs)

    # --- resolve chosen eye video files ---
    rv_raw = _pick_first_video(_require_attr(block, "re_videos"), "re_videos")
    lv_raw = _pick_first_video(_require_attr(block, "le_videos"), "le_videos")
    rv = _resolve_eye_video(rv_raw, eye_video_mode)
    lv = _resolve_eye_video(lv_raw, eye_video_mode)

    # arena path
    arena_list = _require_attr(block, "arena_videos")
    arena_path = _choose_arena_video(arena_list, arena_video)

    # open caps
    capR = _open_cap(rv, "right_eye")
    capL = _open_cap(lv, "left_eye")
    capA = _open_cap(arena_path, "arena")

    Wr, Hr = int(capR.get(cv2.CAP_PROP_FRAME_WIDTH)), int(capR.get(cv2.CAP_PROP_FRAME_HEIGHT))
    Wl, Hl = int(capL.get(cv2.CAP_PROP_FRAME_WIDTH)), int(capL.get(cv2.CAP_PROP_FRAME_HEIGHT))
    Wa, Ha = int(capA.get(cv2.CAP_PROP_FRAME_WIDTH)), int(capA.get(cv2.CAP_PROP_FRAME_HEIGHT))

    dt_ms = 1000.0 / float(fps)
    t_grid = np.arange(start_ms, end_ms + 0.5 * dt_ms, dt_ms, dtype=float)

    # -------------------------- trace mapping --------------------------
    default_map = {
        "pupil_diameter": "pupil_diameter",
        "phi": "phi",
        "theta": "k_theta" if ("k_theta" in left_df.columns and "k_theta" in right_df.columns) else "theta",
    }
    if trace_col_map is None:
        trace_col_map = default_map
    else:
        tmp = default_map.copy()
        tmp.update(trace_col_map)
        trace_col_map = tmp

    # Precompute trace arrays on the export timebase
    Lsig: Dict[str, np.ndarray] = {}
    Rsig: Dict[str, np.ndarray] = {}
    for sig in trace_signals:
        col = trace_col_map.get(sig, sig)
        if col not in left_df.columns or col not in right_df.columns:
            Lsig[sig] = np.full_like(t_grid, np.nan, dtype=float)
            Rsig[sig] = np.full_like(t_grid, np.nan, dtype=float)
            continue

        Lvals = np.full_like(t_grid, np.nan, dtype=float)
        Rvals = np.full_like(t_grid, np.nan, dtype=float)
        for i, t in enumerate(t_grid):
            rL = _nearest_row_by_ms(left_df, t)
            rR = _nearest_row_by_ms(right_df, t)
            vL = rL.get(col, np.nan)
            vR = rR.get(col, np.nan)
            Lvals[i] = float(vL) if pd.notna(vL) else np.nan
            Rvals[i] = float(vR) if pd.notna(vR) else np.nan
        Lsig[sig] = Lvals
        Rsig[sig] = Rvals

    # -------------------------- output geometry --------------------------
    if fit_eyes_to_arena_height:
        Wr_out = max(1, int(round(Wr * (Ha / float(Hr)))))
        Wl_out = max(1, int(round(Wl * (Ha / float(Hl)))))
        Wa_out = Wa
        Hrow = Ha
    else:
        Wr_out, Wl_out, Wa_out = Wr, Wl, Wa
        Hrow = max(Hr, Hl, Ha)

    Wtotal = Wr_out + Wa_out + Wl_out
    Htotal = top_banner_h + Hrow + trace_h

    out_path = Path(out_path)
    out_path.parent.mkdir(parents=True, exist_ok=True)

    writer = cv2.VideoWriter(
        str(out_path),
        cv2.VideoWriter_fourcc(*codec),
        float(fps),
        (Wtotal, Htotal),
    )
    if not writer.isOpened():
        for cap in (capR, capL, capA):
            cap.release()
        raise RuntimeError(f"Could not open VideoWriter for: {out_path} (codec='{codec}')")

    # banner (include eye_video_mode for provenance)
    animal = getattr(block, "animal_call", None) or getattr(block, "animal", None) or ""
    blk = getattr(block, "block", None) or getattr(block, "block_num", None) or ""
    banner_lines = [
        f"Synchronized montage | {animal} {('B'+str(blk)) if str(blk) else ''}".strip(),
        f"eye_video_mode={eye_video_mode} | Right: {rv.name} | Arena: {arena_path.name} | Left: {lv.name} | arena_shift={int(arena_frame_shift)}",
    ]
    banner = _make_banner(Wtotal, banner_lines)

    if show_debug_prints:
        print(f"[export] output={Wtotal}x{Htotal} @ {fps} fps | frames={len(t_grid)}")
        print(f"[eyes] mode={eye_video_mode} | R={rv.name} | L={lv.name}")
        print(f"[arena] {arena_path.name} | arena_frame_shift={int(arena_frame_shift)}")

    # last-good frames for duplication on read failure
    prev_R = np.zeros((Hr, Wr, 3), dtype=np.uint8)
    prev_L = np.zeros((Hl, Wl, 3), dtype=np.uint8)
    prev_A = np.zeros((Ha, Wa, 3), dtype=np.uint8)

    try:
        for t_ms in tqdm(t_grid, desc="Exporting montage", unit="frame", dynamic_ncols=True):
            # nearest eye rows → eye frames
            rR = _nearest_row_by_ms(right_df, t_ms)
            rL = _nearest_row_by_ms(left_df, t_ms)
            idxR = int(rR["eye_frame"]) if pd.notna(rR["eye_frame"]) else None
            idxL = int(rL["eye_frame"]) if pd.notna(rL["eye_frame"]) else None

            # arena frame index
            if use_final_sync_df:
                rA = _nearest_row_by_ms(fs, t_ms)
                vA = rA.get(arena_fcol, pd.NA)
                idxA = int(vA) if pd.notna(vA) else None
            else:
                fpsA = capA.get(cv2.CAP_PROP_FPS) or fps
                idxA = int(round((t_ms - start_ms) / 1000.0 * float(fpsA)))

            # manual patch
            if idxA is not None:
                idxA = int(idxA) + int(arena_frame_shift)

            # clamp
            idxR = _clamp_idx(idxR, capR)
            idxL = _clamp_idx(idxL, capL)
            idxA = _clamp_idx(idxA, capA)

            # read with fallback (duplicate previous if missing)
            missingR = missingL = missingA = False

            fR = _read_frame_at(capR, idxR) if idxR is not None else None
            if fR is None:
                fR = prev_R.copy()
                missingR = True
            else:
                prev_R = fR.copy()

            fL = _read_frame_at(capL, idxL) if idxL is not None else None
            if fL is None:
                fL = prev_L.copy()
                missingL = True
            else:
                prev_L = fL.copy()

            fA = _read_frame_at(capA, idxA) if idxA is not None else None
            if fA is None:
                fA = prev_A.copy()
                missingA = True
            else:
                prev_A = fA.copy()

            # flips
            if flip_eyes_vertical:
                fR = cv2.flip(fR, 0)
                fL = cv2.flip(fL, 0)

            # disqualified flags (show but keep frames)
            disqR = any((c in right_df.columns and pd.isna(rR.get(c, np.nan))) for c in disqualify_cols)
            disqL = any((c in left_df.columns and pd.isna(rL.get(c, np.nan))) for c in disqualify_cols)

            # overlays
            _safe_put_text(fR, "RIGHT", (12, 24), (255, 255, 255), scale=0.75, thickness=2)
            _safe_put_text(fA, "ARENA", (12, 24), (255, 255, 255), scale=0.75, thickness=2)
            _safe_put_text(fL, "LEFT",  (12, 24), (255, 255, 255), scale=0.75, thickness=2)

            if show_disqualified_badge and disqR:
                _safe_put_text(fR, "disqualified", (max(10, fR.shape[1] - 170), 24), (0, 0, 255), scale=0.65, thickness=2)
            if show_disqualified_badge and disqL:
                _safe_put_text(fL, "disqualified", (max(10, fL.shape[1] - 170), 24), (0, 0, 255), scale=0.65, thickness=2)

            if missingR:
                _safe_put_text(fR, "missing frame", (12, fR.shape[0] - 12), (255, 0, 0), scale=0.6, thickness=2)
            if missingL:
                _safe_put_text(fL, "missing frame", (12, fL.shape[0] - 12), (255, 0, 0), scale=0.6, thickness=2)
            if missingA:
                _safe_put_text(fA, "missing frame", (12, fA.shape[0] - 12), (255, 0, 0), scale=0.6, thickness=2)

            # timestamp below arena (inside arena image bottom center)
            ts_str = f"{t_ms:.{timestamp_precision_ms}f} ms"
            ts_sz, _ = cv2.getTextSize(ts_str, cv2.FONT_HERSHEY_SIMPLEX, 0.85, 2)
            tx = max(12, (fA.shape[1] - ts_sz[0]) // 2)
            ty = max(28, fA.shape[0] - 12)
            _safe_put_text(fA, ts_str, (tx, ty), (255, 255, 255), scale=0.85, thickness=2)

            # resize/pad into row
            if fit_eyes_to_arena_height:
                fR = _resize_to_height(fR, Ha)
                fL = _resize_to_height(fL, Ha)
                # arena stays native height Ha
            else:
                def _pad(img: np.ndarray, H: int) -> np.ndarray:
                    h, w = img.shape[:2]
                    if h == H:
                        return img
                    return cv2.copyMakeBorder(img, 0, max(0, H - h), 0, 0, cv2.BORDER_CONSTANT, value=(0, 0, 0))

                fR = _pad(fR, Hrow)
                fA = _pad(fA, Hrow)
                fL = _pad(fL, Hrow)

            row_img = np.concatenate([fR, fA, fL], axis=1)

            trace = _draw_trace_panel(
                W=Wtotal,
                t_ms=t_ms,
                t_grid=t_grid,
                Lsig=Lsig,
                Rsig=Rsig,
                t0=start_ms,
                t1=end_ms,
            )

            frame = np.zeros((Htotal, Wtotal, 3), dtype=np.uint8)
            frame[0:top_banner_h, :, :] = banner
            frame[top_banner_h:top_banner_h + Hrow, :, :] = row_img
            frame[top_banner_h + Hrow:top_banner_h + Hrow + trace_h, :, :] = trace

            writer.write(frame)

        return out_path

    finally:
        try:
            writer.release()
        except Exception:
            pass
        for cap in (capR, capL, capA):
            try:
                cap.release()
            except Exception:
                pass


In [32]:
export_block_synchronized_montage_video_v3(
    block, 240_000, 300_000,
    out_path = str(block.analysis_path / '240_300_montage_raw_synced.mp4'),
    eye_video_mode="raw",
    arena_video="top",
    arena_frame_shift=-3,
    fps=60
)


Exporting montage:   0%|          | 0/3601 [00:00<?, ?frame/s]

[export] output=2720x1360 @ 60 fps | frames=3601
[eyes] mode=raw | R=wake3.mp4 | L=wake3_LE.mp4
[arena] top_20240718T124930.mp4 | arena_frame_shift=-3


<ipython-input-31-e3eff2c31886>:237: RuntimeWarning: invalid value encountered in cast
  yL = (yy0 + (1.0 - np.clip(Ln, 0, 1)) * (Hax - 1)).astype(int)
<ipython-input-31-e3eff2c31886>:238: RuntimeWarning: invalid value encountered in cast
  yR = (yy0 + (1.0 - np.clip(Rn, 0, 1)) * (Hax - 1)).astype(int)
Exporting montage: 100%|██████████| 3601/3601 [10:18<00:00,  5.83frame/s]


WindowsPath('Z:/Nimrod/experiments/PV_126/2024_07_18/block_007/analysis/240_300_montage_raw_synced.mp4')

In [20]:
# export synchronized video montage:
from __future__ import annotations

import os
from pathlib import Path
from typing import Optional, Sequence, Dict, Any, Tuple, Union

import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm


def export_block_synchronized_montage_video_v2(
    block: object,
    start_ms: float,
    end_ms: float,
    out_path: os.PathLike | str,
    *,
    fps: float = 60.0,
    use_final_sync_df: bool = True,

    # --- NEW: manual band-aid sync patch ---
    arena_frame_shift: int = 0,  # + shifts arena later (uses later frames); - shifts earlier

    # Arena selection
    arena_video: Optional[Union[int, str]] = None,  # None => prompt; int => index; str => substring match
    arena_frame_cols: Sequence[str] = (
        "Arena_frame", "arena_frame", "arena_frames", "arena_frame_idx",
        "frame", "frame_idx", "video_frame", "arena_idx"
    ),

    # Eye video selection/transform
    prefer_dlc_eye_videos: bool = True,
    flip_eyes_vertical: bool = True,

    # Layout sizing
    top_banner_h: int = 60,
    trace_h: int = 220,
    fit_eyes_to_arena_height: bool = False,  # scales eye panels to arena height, preserve aspect ratio

    # Trace controls
    trace_window_ms: Optional[float] = None,   # None = full clip, else rolling window
    normalize_traces: bool = False,            # default OFF (native units)
    robust_ylim_percentiles: Tuple[float, float] = (5.0, 95.0),
    trace_signals: Sequence[str] = ("pupil_diameter", "phi", "theta"),
    trace_col_map: Optional[Dict[str, str]] = None,  # mapping display->df column

    # Disqualification logic + missing frames
    disqualify_cols: Sequence[str] = ("center_x", "center_y"),
    show_disqualified_badge: bool = True,

    timestamp_precision_ms: int = 0,
    codec: str = "mp4v",
    show_debug_prints: bool = True,
) -> Path:
    """
    Export a synchronized montage video:

      [Right eye | Arena | Left eye]
      + top banner (text)
      + bottom trace panel with pupil diameter + phi/theta (cursor at current t)

    Manual sync patch:
      - arena_frame_shift: constant integer offset applied to arena frame indices
        before reading frames from the arena video file.

        If LED appears EARLIER in arena vs eyes, you usually want arena_frame_shift=+N
        (use later arena frames so the blink occurs later in arena).
    """

    # -------------------------- helpers --------------------------
    def _require_attr(obj: object, name: str):
        if not hasattr(obj, name):
            raise AttributeError(f"block is missing required attribute '{name}'")
        return getattr(obj, name)

    def _require_cols(df: pd.DataFrame, cols: Sequence[str], df_name: str):
        missing = [c for c in cols if c not in df.columns]
        if missing:
            raise ValueError(f"{df_name} is missing required columns: {missing}")

    def _pick_first_video(paths: Any, name: str) -> Path:
        if paths is None:
            raise ValueError(f"block.{name} is None")
        if isinstance(paths, (str, os.PathLike)):
            p = Path(paths)
            if not p.exists():
                raise FileNotFoundError(f"Video not found: {p}")
            return p
        if isinstance(paths, (list, tuple)) and len(paths) > 0:
            p = Path(paths[0])
            if not p.exists():
                raise FileNotFoundError(f"Video not found: {p}")
            return p
        raise ValueError(f"block.{name} has no usable video path(s)")

    def _find_dlc_variant(original: Path) -> Optional[Path]:
        """Heuristic: same folder, *.mp4 with 'DLC' in name; prefer those containing original stem."""
        try:
            folder = original.parent
            if not folder.exists():
                return None
            cand = [p for p in folder.glob("*.mp4") if "dlc" in p.name.lower()]
            if not cand:
                return None
            stem = original.stem.lower()
            cand2 = [p for p in cand if stem in p.stem.lower()]
            best = cand2[0] if cand2 else cand[0]
            return best if best.exists() else None
        except Exception:
            return None

    def _open_cap(p: Path, label: str) -> cv2.VideoCapture:
        cap = cv2.VideoCapture(str(p))
        if not cap.isOpened():
            raise RuntimeError(f"Cannot open {label} video: {p}")
        return cap

    def _nearest_row_by_ms(df: pd.DataFrame, ms: float) -> pd.Series:
        arr = df["ms_axis"].to_numpy(dtype=float)
        if arr.size == 0:
            raise ValueError("ms_axis array is empty.")
        idx = int(np.nanargmin(np.abs(arr - float(ms))))
        return df.iloc[idx]

    def _resolve_arena_frame_col(fs: pd.DataFrame) -> str:
        for c in arena_frame_cols:
            if c in fs.columns:
                return c
        raise ValueError(
            f"final_sync_df has no recognizable arena frame column. Tried: {list(arena_frame_cols)}"
        )

    def _safe_put_text(img, text, org, color, scale=0.6, thickness=2):
        x, y = org
        cv2.putText(img, text, (x + 1, y + 1), cv2.FONT_HERSHEY_SIMPLEX, scale, (0, 0, 0),
                    thickness + 2, cv2.LINE_AA)
        cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_SIMPLEX, scale, color, thickness, cv2.LINE_AA)

    def _read_frame_at(cap: cv2.VideoCapture, frame_idx: int) -> Optional[np.ndarray]:
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(frame_idx))
        ok, frame = cap.read()
        return frame if ok else None

    def _make_banner(W: int, lines: Sequence[str]) -> np.ndarray:
        banner = np.zeros((top_banner_h, W, 3), dtype=np.uint8)
        y = 24
        for ln in lines:
            _safe_put_text(banner, ln, (12, y), (255, 255, 255), scale=0.7, thickness=2)
            y += 24
            if y > top_banner_h - 8:
                break
        return banner

    def _choose_arena_video(arena_list: Sequence[str], choice: Optional[Union[int, str]]) -> Path:
        paths = [Path(p) for p in arena_list]
        if not paths:
            raise ValueError("block.arena_videos is empty.")

        if isinstance(choice, int):
            idx = int(choice)
            if idx < 0 or idx >= len(paths):
                raise ValueError(f"arena_video index {idx} out of range (0..{len(paths)-1})")
            return paths[idx]

        if isinstance(choice, str) and choice.strip():
            key = choice.strip().lower()
            hits = [p for p in paths if key in p.name.lower()]
            if len(hits) == 1:
                return hits[0]
            if len(hits) > 1:
                hits.sort(key=lambda p: len(p.name))
                return hits[0]
            raise ValueError(f"arena_video='{choice}' did not match any arena video name.")

        print("\nSelect arena video:")
        for i, p in enumerate(paths):
            print(f"  [{i}] {p.name}")
        try:
            raw = input("Enter arena index (default 0): ").strip()
        except Exception:
            raw = ""
        idx = 0 if raw == "" else int(raw)
        if idx < 0 or idx >= len(paths):
            raise ValueError(f"arena index {idx} out of range.")
        return paths[idx]

    def _resize_to_height(img: np.ndarray, target_h: int) -> np.ndarray:
        h, w = img.shape[:2]
        if h == target_h:
            return img
        new_w = max(1, int(round(w * (target_h / float(h)))))
        return cv2.resize(img, (new_w, target_h), interpolation=cv2.INTER_AREA)

    def _robust_limits(x: np.ndarray, p_lo: float, p_hi: float) -> Tuple[float, float]:
        x = np.asarray(x, dtype=float)
        x = x[np.isfinite(x)]
        if x.size < 2:
            return (-1.0, 1.0)
        lo = np.percentile(x, p_lo)
        hi = np.percentile(x, p_hi)
        if not np.isfinite(lo) or not np.isfinite(hi) or abs(hi - lo) < 1e-12:
            lo, hi = float(np.min(x)), float(np.max(x))
        if abs(hi - lo) < 1e-12:
            lo -= 1.0
            hi += 1.0
        return float(lo), float(hi)

    def _map_to_axis(vals: np.ndarray, lo: float, hi: float) -> np.ndarray:
        return (vals - lo) / (hi - lo + 1e-12)

    def _draw_trace_panel(
        W: int,
        t_ms: float,
        t_grid: np.ndarray,
        Lsig: Dict[str, np.ndarray],
        Rsig: Dict[str, np.ndarray],
        t0: float,
        t1: float,
    ) -> np.ndarray:
        panel = np.zeros((trace_h, W, 3), dtype=np.uint8)

        if trace_window_ms is None:
            w0, w1 = t0, t1
        else:
            half = float(trace_window_ms) / 2.0
            w0, w1 = max(t0, t_ms - half), min(t1, t_ms + half)
            if w1 <= w0:
                w0, w1 = t0, t1

        idx = np.where((t_grid >= w0) & (t_grid <= w1))[0]
        if idx.size < 2:
            return panel

        tg = t_grid[idx]
        x = (tg - w0) / (w1 - w0 + 1e-12)
        xpix = (x * (W - 1)).astype(int)

        n_axes = len(trace_signals)
        pad_y = 10
        axis_h = max(40, (trace_h - 2 * pad_y) // max(1, n_axes))
        axes = []
        for j, name in enumerate(trace_signals):
            y0 = pad_y + j * axis_h
            y1 = min(trace_h - pad_y, y0 + axis_h) - 8
            axes.append((name, y0, y1))

        cursor_x = int(round((np.clip(t_ms, w0, w1) - w0) / (w1 - w0 + 1e-12) * (W - 1)))
        cv2.line(panel, (cursor_x, 0), (cursor_x, trace_h - 1), (120, 120, 120), 1)

        _safe_put_text(panel, f"t = {t_ms:.{timestamp_precision_ms}f} ms", (12, 22),
                       (255, 255, 255), scale=0.65, thickness=2)

        color_L = (80, 220, 80)    # green
        color_R = (220, 220, 80)   # yellow

        for name, y0, y1 in axes:
            cv2.rectangle(panel, (0, y0), (W - 1, y1), (20, 20, 20), 1)

            L = Lsig.get(name, None)
            R = Rsig.get(name, None)
            if L is None or R is None:
                continue

            Lw = L[idx].astype(float)
            Rw = R[idx].astype(float)

            yy0, yy1 = int(y0 + 18), int(y1 - 8)
            Hax = max(2, yy1 - yy0)

            if normalize_traces:
                Llo, Lhi = np.nanmin(Lw), np.nanmax(Lw)
                Rlo, Rhi = np.nanmin(Rw), np.nanmax(Rw)
                if not np.isfinite(Llo) or not np.isfinite(Lhi) or abs(Lhi - Llo) < 1e-12:
                    Ln = np.full_like(Lw, 0.5)
                else:
                    Ln = _map_to_axis(Lw, Llo, Lhi)
                if not np.isfinite(Rlo) or not np.isfinite(Rhi) or abs(Rhi - Rlo) < 1e-12:
                    Rn = np.full_like(Rw, 0.5)
                else:
                    Rn = _map_to_axis(Rw, Rlo, Rhi)
                label = f"{name} (norm)"
            else:
                p_lo, p_hi = robust_ylim_percentiles
                lo, hi = _robust_limits(np.concatenate([Lw, Rw]), p_lo, p_hi)
                Ln = _map_to_axis(Lw, lo, hi)
                Rn = _map_to_axis(Rw, lo, hi)
                label = f"{name} [{lo:.2f},{hi:.2f}]"

            yL = (yy0 + (1.0 - np.clip(Ln, 0, 1)) * (Hax - 1)).astype(int)
            yR = (yy0 + (1.0 - np.clip(Rn, 0, 1)) * (Hax - 1)).astype(int)

            for k in range(1, len(xpix)):
                if np.isfinite(yL[k - 1]) and np.isfinite(yL[k]):
                    cv2.line(panel, (xpix[k - 1], yL[k - 1]), (xpix[k], yL[k]), color_L, 1, cv2.LINE_AA)
                if np.isfinite(yR[k - 1]) and np.isfinite(yR[k]):
                    cv2.line(panel, (xpix[k - 1], yR[k - 1]), (xpix[k], yR[k]), color_R, 1, cv2.LINE_AA)

            _safe_put_text(panel, label, (12, y0 + 16), (200, 200, 200), scale=0.55, thickness=1)
            _safe_put_text(panel, "L", (W - 60, y0 + 16), color_L, scale=0.55, thickness=1)
            _safe_put_text(panel, "R", (W - 35, y0 + 16), color_R, scale=0.55, thickness=1)

        return panel

    def _clamp_idx(idx: Optional[int], cap: cv2.VideoCapture) -> Optional[int]:
        if idx is None:
            return None
        n = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
        if n > 0:
            return int(np.clip(int(idx), 0, n - 1))
        return max(0, int(idx))

    # -------------------------- validation --------------------------
    start_ms = float(start_ms)
    end_ms = float(end_ms)
    if end_ms <= start_ms:
        raise ValueError(f"end_ms must be > start_ms (got start_ms={start_ms}, end_ms={end_ms})")

    left_df: pd.DataFrame = _require_attr(block, "left_eye_data")
    right_df: pd.DataFrame = _require_attr(block, "right_eye_data")
    _require_cols(left_df, ["ms_axis", "eye_frame"], "left_eye_data")
    _require_cols(right_df, ["ms_axis", "eye_frame"], "right_eye_data")

    fs = None
    arena_fcol = None
    if use_final_sync_df:
        fs = _require_attr(block, "final_sync_df")
        if not isinstance(fs, pd.DataFrame):
            raise ValueError("block.final_sync_df is not a pandas DataFrame.")
        _require_cols(fs, ["ms_axis"], "final_sync_df")
        arena_fcol = _resolve_arena_frame_col(fs)

    rv = _pick_first_video(_require_attr(block, "re_videos"), "re_videos")
    lv = _pick_first_video(_require_attr(block, "le_videos"), "le_videos")

    if prefer_dlc_eye_videos:
        rv_dlc = _find_dlc_variant(rv)
        lv_dlc = _find_dlc_variant(lv)
        if rv_dlc is not None:
            if show_debug_prints:
                print(f"[video] Using RIGHT DLC: {rv_dlc.name}")
            rv = rv_dlc
        if lv_dlc is not None:
            if show_debug_prints:
                print(f"[video] Using LEFT DLC:  {lv_dlc.name}")
            lv = lv_dlc

    arena_list = _require_attr(block, "arena_videos")
    arena_path = _choose_arena_video(arena_list, arena_video)

    capR = _open_cap(rv, "right_eye")
    capL = _open_cap(lv, "left_eye")
    capA = _open_cap(arena_path, "arena")

    Wr, Hr = int(capR.get(cv2.CAP_PROP_FRAME_WIDTH)), int(capR.get(cv2.CAP_PROP_FRAME_HEIGHT))
    Wl, Hl = int(capL.get(cv2.CAP_PROP_FRAME_WIDTH)), int(capL.get(cv2.CAP_PROP_FRAME_HEIGHT))
    Wa, Ha = int(capA.get(cv2.CAP_PROP_FRAME_WIDTH)), int(capA.get(cv2.CAP_PROP_FRAME_HEIGHT))

    dt_ms = 1000.0 / float(fps)
    t_grid = np.arange(start_ms, end_ms + 0.5 * dt_ms, dt_ms, dtype=float)

    # -------------------------- trace column mapping --------------------------
    default_map = {
        "pupil_diameter": "pupil_diameter",
        "phi": "phi",
        "theta": "k_theta" if ("k_theta" in left_df.columns and "k_theta" in right_df.columns) else "theta",
    }
    if trace_col_map is None:
        trace_col_map = default_map
    else:
        tmp = default_map.copy()
        tmp.update(trace_col_map)
        trace_col_map = tmp

    # Precompute trace signals sampled on the same output timebase
    Lsig: Dict[str, np.ndarray] = {}
    Rsig: Dict[str, np.ndarray] = {}
    for sig in trace_signals:
        col = trace_col_map.get(sig, sig)
        if col not in left_df.columns or col not in right_df.columns:
            if show_debug_prints:
                print(f"[warn] Trace '{sig}' requested but column '{col}' not found in both eyes; trace will be blank.")
            Lsig[sig] = np.full_like(t_grid, np.nan, dtype=float)
            Rsig[sig] = np.full_like(t_grid, np.nan, dtype=float)
            continue

        Lvals = np.full_like(t_grid, np.nan, dtype=float)
        Rvals = np.full_like(t_grid, np.nan, dtype=float)
        for i, t in enumerate(t_grid):
            rL = _nearest_row_by_ms(left_df, t)
            rR = _nearest_row_by_ms(right_df, t)
            vL = rL.get(col, np.nan)
            vR = rR.get(col, np.nan)
            Lvals[i] = float(vL) if pd.notna(vL) else np.nan
            Rvals[i] = float(vR) if pd.notna(vR) else np.nan
        Lsig[sig] = Lvals
        Rsig[sig] = Rvals

    # -------------------------- output geometry --------------------------
    if fit_eyes_to_arena_height:
        Wr_out = max(1, int(round(Wr * (Ha / float(Hr)))))
        Wl_out = max(1, int(round(Wl * (Ha / float(Hl)))))
        Wa_out, Ha_out = Wa, Ha
        Hrow = Ha
    else:
        Wr_out, Wl_out, Wa_out = Wr, Wl, Wa
        Ha_out = Ha
        Hrow = max(Hr, Hl, Ha)

    Wtotal = Wr_out + Wa_out + Wl_out
    Htotal = top_banner_h + Hrow + trace_h

    out_path = Path(out_path)
    out_path.parent.mkdir(parents=True, exist_ok=True)

    fourcc = cv2.VideoWriter_fourcc(*codec)
    writer = cv2.VideoWriter(str(out_path), fourcc, float(fps), (Wtotal, Htotal))
    if not writer.isOpened():
        capR.release(); capL.release(); capA.release()
        raise RuntimeError(f"Could not open VideoWriter for: {out_path} (codec='{codec}')")

    animal = getattr(block, "animal_call", None) or getattr(block, "animal", None) or ""
    blk = getattr(block, "block", None) or getattr(block, "block_num", None) or ""
    banner_lines = [
        f"Synchronized montage | {animal} {('B'+str(blk)) if str(blk) else ''}".strip(),
        f"Right: {rv.name} | Arena: {arena_path.name} | Left: {lv.name} | arena_frame_shift={int(arena_frame_shift)}",
    ]
    banner = _make_banner(Wtotal, banner_lines)

    if show_debug_prints:
        print(f"[export] frames={len(t_grid)} | fps={fps} | dt_ms={dt_ms:.3f}")
        print(f"[export] output={Wtotal}x{Htotal} | rowH={Hrow} | fit_eyes_to_arena_height={fit_eyes_to_arena_height}")
        if use_final_sync_df:
            print(f"[arena] final_sync_df arena frame col='{arena_fcol}'")
        print(f"[arena] arena_frame_shift={int(arena_frame_shift)}")

    prev_R = np.zeros((Hr, Wr, 3), dtype=np.uint8)
    prev_L = np.zeros((Hl, Wl, 3), dtype=np.uint8)
    prev_A = np.zeros((Ha, Wa, 3), dtype=np.uint8)

    # -------------------------- main loop --------------------------
    try:
        for t_ms in tqdm(
            t_grid,
            desc="Exporting montage",
            unit="frame",
            total=len(t_grid),
            dynamic_ncols=True,
            smoothing=0.1,
        ):
            rR = _nearest_row_by_ms(right_df, t_ms)
            rL = _nearest_row_by_ms(left_df, t_ms)

            idxR = int(rR["eye_frame"]) if pd.notna(rR["eye_frame"]) else None
            idxL = int(rL["eye_frame"]) if pd.notna(rL["eye_frame"]) else None

            if use_final_sync_df:
                rA = _nearest_row_by_ms(fs, t_ms)
                vA = rA.get(arena_fcol, pd.NA)
                idxA = int(vA) if pd.notna(vA) else None
            else:
                fpsA = capA.get(cv2.CAP_PROP_FPS) or fps
                idxA = int(round((t_ms - start_ms) / 1000.0 * float(fpsA)))

            # --- NEW: apply manual arena shift ---
            if idxA is not None:
                idxA = int(idxA) + int(arena_frame_shift)

            # clamp indices so we don't seek negative / beyond end
            idxR = _clamp_idx(idxR, capR)
            idxL = _clamp_idx(idxL, capL)
            idxA = _clamp_idx(idxA, capA)

            missingR = missingL = missingA = False

            fR = _read_frame_at(capR, idxR) if idxR is not None else None
            if fR is None:
                fR = prev_R.copy()
                missingR = True
            else:
                prev_R = fR.copy()

            fL = _read_frame_at(capL, idxL) if idxL is not None else None
            if fL is None:
                fL = prev_L.copy()
                missingL = True
            else:
                prev_L = fL.copy()

            fA = _read_frame_at(capA, idxA) if idxA is not None else None
            if fA is None:
                fA = prev_A.copy()
                missingA = True
            else:
                prev_A = fA.copy()

            if flip_eyes_vertical:
                fR = cv2.flip(fR, 0)
                fL = cv2.flip(fL, 0)

            disqR = False
            disqL = False
            for c in disqualify_cols:
                if c in right_df.columns and pd.isna(rR.get(c, np.nan)):
                    disqR = True
                if c in left_df.columns and pd.isna(rL.get(c, np.nan)):
                    disqL = True

            _safe_put_text(fR, "RIGHT", (12, 24), (255, 255, 255), scale=0.75, thickness=2)
            _safe_put_text(fA, "ARENA", (12, 24), (255, 255, 255), scale=0.75, thickness=2)
            _safe_put_text(fL, "LEFT",  (12, 24), (255, 255, 255), scale=0.75, thickness=2)

            if show_disqualified_badge and disqR:
                _safe_put_text(fR, "disqualified", (max(10, fR.shape[1] - 170), 24), (0, 0, 255), scale=0.65, thickness=2)
            if show_disqualified_badge and disqL:
                _safe_put_text(fL, "disqualified", (max(10, fL.shape[1] - 170), 24), (0, 0, 255), scale=0.65, thickness=2)

            if missingR:
                _safe_put_text(fR, "missing frame", (12, fR.shape[0] - 12), (255, 0, 0), scale=0.6, thickness=2)
            if missingL:
                _safe_put_text(fL, "missing frame", (12, fL.shape[0] - 12), (255, 0, 0), scale=0.6, thickness=2)
            if missingA:
                _safe_put_text(fA, "missing frame", (12, fA.shape[0] - 12), (255, 0, 0), scale=0.6, thickness=2)

            ts_str = f"{t_ms:.{timestamp_precision_ms}f} ms"
            ts_sz, _ = cv2.getTextSize(ts_str, cv2.FONT_HERSHEY_SIMPLEX, 0.85, 2)
            tx = max(12, (fA.shape[1] - ts_sz[0]) // 2)
            ty = max(28, fA.shape[0] - 12)
            _safe_put_text(fA, ts_str, (tx, ty), (255, 255, 255), scale=0.85, thickness=2)

            if fit_eyes_to_arena_height:
                fR = _resize_to_height(fR, Ha)
                fL = _resize_to_height(fL, Ha)
                # arena stays native (Ha)
            else:
                # pad to common row height
                def _pad(img: np.ndarray, H: int) -> np.ndarray:
                    h, w = img.shape[:2]
                    if h == H:
                        return img
                    return cv2.copyMakeBorder(img, 0, max(0, H - h), 0, 0, cv2.BORDER_CONSTANT, value=(0, 0, 0))

                fR = _pad(fR, Hrow)
                fA = _pad(fA, Hrow)
                fL = _pad(fL, Hrow)

            row_img = np.concatenate([fR, fA, fL], axis=1)

            trace = _draw_trace_panel(
                W=Wtotal,
                t_ms=t_ms,
                t_grid=t_grid,
                Lsig=Lsig,
                Rsig=Rsig,
                t0=start_ms,
                t1=end_ms,
            )

            frame = np.zeros((Htotal, Wtotal, 3), dtype=np.uint8)
            frame[0:top_banner_h, :, :] = banner
            frame[top_banner_h:top_banner_h + Hrow, :, :] = row_img
            frame[top_banner_h + Hrow:top_banner_h + Hrow + trace_h, :, :] = trace

            writer.write(frame)

        return out_path

    finally:
        try:
            writer.release()
        except Exception:
            pass
        for cap in (capR, capL, capA):
            try:
                cap.release()
            except Exception:
                pass


In [23]:
export_block_synchronized_montage_video_v2(
    block=block,
    start_ms=240_000, end_ms=320_000,out_path = str(block.analysis_path / '240_320_montage_synced.mp4'),
    fps= 60.0,
    # Arena selection
    arena_video = 'top',
    arena_frame_shift=-3)

[video] Using RIGHT DLC: wake3DLC_resnet_50_Eye_Tracking_piplineMar1shuffle1_950000_labeled.mp4
[video] Using LEFT DLC:  wake3_LEDLC_resnet_50_Eye_Tracking_piplineMar1shuffle1_950000_labeled_LE.mp4


Exporting montage:   0%|          | 0/4801 [00:00<?, ?frame/s]

[export] frames=4801 | fps=60.0 | dt_ms=16.667
[export] output=2720x1360 | rowH=1080 | fit_eyes_to_arena_height=False
[arena] final_sync_df arena frame col='Arena_frame'
[arena] arena_frame_shift=-3


<ipython-input-20-33202e265489>:291: RuntimeWarning: invalid value encountered in cast
  yL = (yy0 + (1.0 - np.clip(Ln, 0, 1)) * (Hax - 1)).astype(int)
<ipython-input-20-33202e265489>:292: RuntimeWarning: invalid value encountered in cast
  yR = (yy0 + (1.0 - np.clip(Rn, 0, 1)) * (Hax - 1)).astype(int)
Exporting montage: 100%|██████████| 4801/4801 [14:05<00:00,  5.68frame/s]


WindowsPath('Z:/Nimrod/experiments/PV_126/2024_07_18/block_007/analysis/240_320_montage_synced.mp4')

In [27]:
import re
import subprocess
from pathlib import Path
from typing import Literal, Optional


def compress_video_verbose(
    in_path: str | Path,
    out_path: Optional[str | Path] = None,
    *,
    preset: Literal[
        "lossless",
        "visually_lossless",
        "publication",
        "web",
        "aggressive",
    ] = "publication",
    overwrite: bool = False,
    keep_audio: bool = True,
    threads: int = 0,
) -> Path:
    """
    ffmpeg video compression with:
      - clear readouts (input/output + settings)
      - ffmpeg built-in progress (frame/fps/time/speed)
      - (optional) overwrite behavior

    Requires: ffmpeg in PATH.
    """

    in_path = Path(in_path)
    if not in_path.exists():
        raise FileNotFoundError(in_path)

    if out_path is None:
        out_path = in_path.with_name(in_path.stem + f"__{preset}.mp4")
    else:
        out_path = Path(out_path)

    if out_path.exists() and not overwrite:
        raise FileExistsError(out_path)

    # -------- preset config --------
    if preset == "lossless":
        codec, crf, enc_preset = "libx264", "0", "veryslow"
    elif preset == "visually_lossless":
        codec, crf, enc_preset = "libx264", "16", "slow"
    elif preset == "publication":
        codec, crf, enc_preset = "libx264", "18", "slow"
    elif preset == "web":
        codec, crf, enc_preset = "libx264", "23", "medium"
    elif preset == "aggressive":
        codec, crf, enc_preset = "libx265", "28", "slow"
    else:
        raise ValueError(f"Unknown preset: {preset}")

    print("\n=== Video compression ===")
    print(f"Input : {in_path}")
    print(f"Output: {out_path}")
    print(f"Preset: {preset} | codec={codec} | crf={crf} | preset={enc_preset}")
    print(f"Audio : {'copy' if keep_audio else 'disabled'}")
    print(f"Overwrite: {overwrite}")
    if threads > 0:
        print(f"Threads: {threads}")
    print("-------------------------")

    cmd = [
        "ffmpeg",
        "-hide_banner",
        "-nostdin",
        "-stats",                 # prints frame/fps/size/time/bitrate/speed
        "-i", str(in_path),
        "-map", "0:v:0",
        "-c:v", codec,
        "-crf", crf,
        "-preset", enc_preset,
    ]

    if keep_audio:
        cmd += ["-map", "0:a?", "-c:a", "copy"]
    else:
        cmd += ["-an"]

    if threads > 0:
        cmd += ["-threads", str(threads)]

    cmd += ["-y" if overwrite else "-n", str(out_path)]

    print("Running ffmpeg command:")
    print("  " + " ".join(cmd))
    print("\nProgress (ffmpeg):")

    # ffmpeg prints progress primarily to stderr; stream it live
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
        bufsize=1,
        universal_newlines=True,
    )

    # Stream stderr lines and keep only the changing "stats" line readable
    last_stats = ""
    try:
        assert proc.stderr is not None
        for line in proc.stderr:
            line = line.rstrip("\n")
            # ffmpeg -stats produces lines starting with "frame="
            if line.strip().startswith("frame="):
                last_stats = line.strip()
                print("\r" + last_stats + " " * 10, end="", flush=True)
            else:
                # print non-stats messages on their own lines
                if line.strip():
                    print("\n" + line)
        proc.wait()
    finally:
        print()  # newline after carriage-return progress

    if proc.returncode != 0:
        raise RuntimeError(f"ffmpeg failed with return code {proc.returncode}")

    # Final readout
    in_size = in_path.stat().st_size
    out_size = out_path.stat().st_size if out_path.exists() else None
    if out_size is not None:
        ratio = out_size / in_size if in_size else float("nan")
        print("\nDone.")
        print(f"Output written: {out_path}")
        print(f"Size: {in_size/1e6:.1f} MB -> {out_size/1e6:.1f} MB  (ratio {ratio:.3f})")
    else:
        print("\nDone (but output file not found?).")

    return out_path


In [33]:
compress_video_verbose(
    str(block.analysis_path / '240_300_montage_raw_synced.mp4'),
    preset="publication",
    overwrite=False
)



=== Video compression ===
Input : Z:\Nimrod\experiments\PV_126\2024_07_18\block_007\analysis\240_300_montage_raw_synced.mp4
Output: Z:\Nimrod\experiments\PV_126\2024_07_18\block_007\analysis\240_300_montage_raw_synced__publication.mp4
Preset: publication | codec=libx264 | crf=18 | preset=slow
Audio : copy
Overwrite: False
-------------------------
Running ffmpeg command:
  ffmpeg -hide_banner -nostdin -stats -i Z:\Nimrod\experiments\PV_126\2024_07_18\block_007\analysis\240_300_montage_raw_synced.mp4 -map 0:v:0 -c:v libx264 -crf 18 -preset slow -map 0:a? -c:a copy -n Z:\Nimrod\experiments\PV_126\2024_07_18\block_007\analysis\240_300_montage_raw_synced__publication.mp4

Progress (ffmpeg):

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'Z:\Nimrod\experiments\PV_126\2024_07_18\block_007\analysis\240_300_montage_raw_synced.mp4':

  Metadata:

    major_brand     : isom

    minor_version   : 512

    compatible_brands: isomiso2mp41

    encoder         : Lavf58.29.100

  Duration: 00:01:00.02, s

WindowsPath('Z:/Nimrod/experiments/PV_126/2024_07_18/block_007/analysis/240_300_montage_raw_synced__publication.mp4')

In [ ]:
# BLOCK DEFINITION #
# This was the previous run
#animals = ['PV_62', 'PV_126', 'PV_57']
#block_lists = [[24, 26, 38], [7, 8, 9, 10, 11, 12], [7, 8, 9, 12, 13]]
#This with new animals:
animals = ['PV_208']
block_lists = [[21]]
experiment_path = pathlib.Path(r"Z:\Nimrod\experiments")
bad_blocks = [0]  # Example of bad blocks

block_collection, block_dict = create_block_collections(
    animals=animals,
    block_lists=block_lists,
    experiment_path=experiment_path,
    bad_blocks=bad_blocks
)
for block in block_collection:
    block.parse_open_ephys_events()
    block.get_eye_brightness_vectors()
    block.synchronize_block()
    block.create_eye_brightness_df(threshold_value=20)

    # if the code fails here, go to manual synchronization
    block.import_manual_sync_df()
    block.read_dlc_data()
    block.calibrate_pixel_size(10)
    #load_eye_data_2d_w_rotation_matrix(block) #should be integrated again... later

    for block in block_collection:
        # block.left_eye_data = pd.read_csv(block.analysis_path / f'left_eye_data_corr_angles.csv')
        # block.right_eye_data = pd.read_csv(block.analysis_path / 'right_eye_data_corr_angles.csv')
        #block.left_eye_data = pd.read_csv(block.analysis_path / f'left_eye_data_degrees_raw_xflipped.csv')
        #block.right_eye_data = pd.read_csv(block.analysis_path / 'right_eye_data_degrees_raw_xflipped.csv')
        block.left_eye_data = pd.read_csv(block.analysis_path / f'left_eye_data_degrees_raw_verified.csv')
        block.right_eye_data = pd.read_csv(block.analysis_path / 'right_eye_data_degrees_raw_verified.csv')
        # block.left_eye_data = pd.read_csv(block.analysis_path / f'left_eye_data_3d_corr_verified.csv')
        # block.right_eye_data = pd.read_csv(block.analysis_path / 'right_eye_data_3d_corr_verified.csv')
        #block.left_eye_data = pd.read_csv(block.analysis_path / f'left_eye_data_degrees_rotated_verified.csv')
        #block.right_eye_data = pd.read_csv(block.analysis_path / 'right_eye_data_degrees_rotated_verified.csv')

    # calibrate pupil diameter:
    # if 'pupil_diameter' not in block.left_eye_data.columns:
    #     block.left_eye_data['pupil_diameter_pixels'] = block.left_eye_data.major_ax * 2 * np.pi
    #     block.right_eye_data['pupil_diameter_pixels'] = block.right_eye_data.major_ax * 2 * np.pi
    #     block.left_eye_data['pupil_diameter'] = block.left_eye_data['pupil_diameter_pixels'] * block.L_pix_size
    #     block.right_eye_data['pupil_diameter'] = block.right_eye_data['pupil_diameter_pixels'] * block.R_pix_size